MIGRACION DE MONGODB A SQLITE3

In [5]:
import sqlite3
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime

# Conectar a MongoDB
mongo_client = MongoClient('localhost', 27017)
mongo_db = mongo_client['db']

# Conectar a SQLite3 y crear una base de datos
sqlite_conn = sqlite3.connect('my_database.db')
sqlite_cursor = sqlite_conn.cursor()

# Crear tablas en SQLite3
sqlite_cursor.execute('''
    CREATE TABLE IF NOT EXISTS countries (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        code TEXT
    )
''')

sqlite_cursor.execute('''
    CREATE TABLE IF NOT EXISTS cities (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT,
        country_id TEXT,
        code TEXT,
        FOREIGN KEY (country_id) REFERENCES countries (id)
    )
''')

sqlite_cursor.execute('''
    CREATE TABLE IF NOT EXISTS prizes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        laurentID TEXT,
        motivation TEXT,
        share INTEGER,
        year INTEGER,
        FOREIGN KEY (laurentID) REFERENCES laureates (id)
    )
''')

sqlite_cursor.execute('''
    CREATE TABLE IF NOT EXISTS laureates (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        firstname TEXT,
        surname TEXT,
        born DATE,
        bornCountry TEXT,
        bornCity TEXT,
        died DATE,
        diedCountry TEXT,
        diedCity TEXT,
        gender TEXT,
        FOREIGN KEY (bornCountry) REFERENCES countries (id),
        FOREIGN KEY (bornCity) REFERENCES cities (id),
        FOREIGN KEY (diedCountry) REFERENCES countries (id),
        FOREIGN KEY (diedCity) REFERENCES cities (id)
    )
''')

# Crea la tabla para afiliaciones
sqlite_cursor.execute('''
    CREATE TABLE IF NOT EXISTS affiliations (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        countryID TEXT,
        cityID TEXT,
        laureate_id TEXT,  -- Cambiado a texto para manejar el ObjectId
        prizeID TEXT,
        name TEXT,
        FOREIGN KEY (laureate_id) REFERENCES laureates (id),
        FOREIGN KEY (countryID) REFERENCES countries (id),
        FOREIGN KEY (cityID) REFERENCES cities (id),
        FOREIGN KEY (prizeID) REFERENCES prizes (id)
    )
''')

# Repite el proceso para otras tablas según sea necesario

# Recuperar datos de MongoDB y insertar en SQLite3
# Función de migración para países
# Función de migración para países

# Función de migración para países
def migrate_countries():
    countries_data = list(mongo_db['countries'].find())

    for country in countries_data:
        # Convertir el campo code a cadena
        country['code'] = str(country.get('code', ''))

        print(f"Inserting data: {country.get('name', '')}, {country['code']}")

        # Insertar datos en la tabla countries (no incluimos el campo id)
        sqlite_cursor.execute('''
            INSERT INTO countries (name, code)
            VALUES (?, ?)
        ''', (country.get('name', ''), country['code']))

def migrate_cities():
    def map_city(document):
        return document

    migrate_collection('cities', 'cities', map_city)

def migrate_prizes():
    migrate_collection('prizes', 'prizes')

def migrate_laureates():
    laureates_data = list(mongo_db['laureates'].find())
    for laureate in laureates_data:
        # Convierte el ObjectId a una cadena antes de insertarlo en SQLite3
        laureate['_id'] = str(laureate['_id'])

        # Migrar datos de ciudades
        city_data = laureate.get('bornCity', {})
        sqlite_cursor.execute('''
            INSERT INTO cities (id, name, country_id, code)
            VALUES (?, ?, ?, ?)
        ''', (str(city_data.get('_id', '')), city_data.get('name', ''), str(city_data.get('country_id', '')), ''))

        # Obtener el ID de la ciudad recién insertada
        sqlite_cursor.execute('SELECT last_insert_rowid()')
        city_id = sqlite_cursor.fetchone()[0]

        # Insertar datos de laureates
        sqlite_cursor.execute('''
            INSERT INTO laureates (id, firstname, surname, born, bornCountry, bornCity, died, diedCountry, diedCity, gender)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            laureate['_id'], laureate['firstname'], laureate['surname'],
            laureate.get('born', None), str(laureate.get('bornCountry', '')),
            str(city_id), laureate.get('died', None), str(laureate.get('diedCountry', '')),
            str(laureate.get('diedCity', '')), laureate.get('gender', '')
        ))

        # Migrar datos de afiliaciones
        affiliations = laureate.get('affiliations', [])
        for affiliation in affiliations:
            sqlite_cursor.execute('''
                INSERT INTO affiliations (id, countryID, cityID, laureate_id, prizeID, name)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (
                str(affiliation.get('_id', '')), str(affiliation.get('countryID', '')),
                str(affiliation.get('cityID', '')), str(laureate['_id']),
                str(affiliation.get('prizeID', '')), affiliation.get('name', '')
            ))

# Repite el proceso para otras colecciones según sea necesario

# Función general para migrar colecciones
def migrate_collection(collection_name, sqlite_table, mapping_function=None):
    mongo_collection = mongo_db[collection_name]
    mongo_data = list(mongo_collection.find())

    for document in mongo_data:
        if mapping_function:
            document = mapping_function(document)

        sqlite_cursor.execute(f'INSERT INTO {sqlite_table} VALUES (?, ?)', tuple(document.values()))

# Llamar a las funciones de migración
migrate_countries()
migrate_cities()
migrate_prizes()
migrate_laureates()

# Guardar cambios y cerrar conexiones
sqlite_conn.commit()
sqlite_conn.close()
mongo_client.close()




TypeError: argument of type 'ObjectId' is not iterable